In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from utils import download_data
from models.neural_network import NeuralNetwork
from environment import Optimizer

In [3]:
train_data, test_data = download_data()

In [4]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=Optimizer.LEARNING_RATE)

In [7]:
def train(dataloader: DataLoader, model, loss_fn: nn.CrossEntropyLoss, optimizer: torch.optim.SGD):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}   [{current:>5d}/{size:>5d}]")


In [8]:
def test(dataloader: DataLoader, model, loss_fn:nn.CrossEntropyLoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg. loss: {test_loss:>8f} ")
    

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------------
loss: 2.297650   [   64/60000]
loss: 2.287532   [ 6464/60000]
loss: 2.271693   [12864/60000]
loss: 2.269808   [19264/60000]
loss: 2.233562   [25664/60000]
loss: 2.214126   [32064/60000]
loss: 2.223717   [38464/60000]
loss: 2.178769   [44864/60000]
loss: 2.189962   [51264/60000]
loss: 2.161560   [57664/60000]
Test Error: 
 Accuracy: 44.0%, Avg. loss: 2.147968 
Epoch 2
------------------------------------
loss: 2.151260   [   64/60000]
loss: 2.146316   [ 6464/60000]
loss: 2.085807   [12864/60000]
loss: 2.112518   [19264/60000]
loss: 2.038376   [25664/60000]
loss: 1.981464   [32064/60000]
loss: 2.023072   [38464/60000]
loss: 1.923222   [44864/60000]
loss: 1.952881   [51264/60000]
loss: 1.874007   [57664/60000]
Test Error: 
 Accuracy: 54.3%, Avg. loss: 1.864073 
Epoch 3
------------------------------------
loss: 1.896618   [   64/60000]
loss: 1.864967   [ 6464/60000]
loss: 1.746310   [12864/60000]
loss: 1.802813   [19264/60000]
loss: 1.664961   